<a href="https://colab.research.google.com/github/Nanda74775/MMO_LAB/blob/main/MMO_LAB6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gym
!pip install numpy
!pip install torch


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical


In [ ]:
class Actor(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.softmax(self.fc2(x), dim=-1)
        return x

class Critic(nn.Module):
    def __init__(self, input_dim):
        super(Critic, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [ ]:
def train(env, actor, critic, actor_optimizer, critic_optimizer, num_episodes=1000, gamma=0.99):
    for episode in range(num_episodes):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        done = False

        while not done:
            state = torch.FloatTensor(state).unsqueeze(0)
            dist = actor(state)
            value = critic(state)

            dist = Categorical(dist)
            action = dist.sample()

            next_state, reward, done, _ = env.step(action.item())

            log_prob = dist.log_prob(action)
            log_probs.append(log_prob)
            values.append(value)
            rewards.append(reward)

            state = next_state

            if done:
                Qval = critic(torch.FloatTensor(next_state).unsqueeze(0)).item()
                returns = []
                R = Qval
                for reward in reversed(rewards):
                    R = reward + gamma * R
                    returns.insert(0, R)

                returns = torch.FloatTensor(returns)
                log_probs = torch.cat(log_probs)
                values = torch.cat(values)

                advantage = returns - values

                actor_loss = - (log_probs * advantage.detach()).mean()
                critic_loss = advantage.pow(2).mean()

                actor_optimizer.zero_grad()
                critic_optimizer.zero_grad()
                actor_loss.backward()
                critic_loss.backward()
                actor_optimizer.step()
                critic_optimizer.step()

                print(f'Episode {episode}, Actor Loss: {actor_loss.item()}, Critic Loss: {critic_loss.item()}')
                break


In [ ]:
def main():
    env = gym.make('CartPole-v1')
    input_dim = env.observation_space.shape[0]
    output_dim = env.action_space.n

    actor = Actor(input_dim, output_dim)
    critic = Critic(input_dim)

    actor_optimizer = optim.Adam(actor.parameters(), lr=1e-3)
    critic_optimizer = optim.Adam(critic.parameters(), lr=1e-3)

    train(env, actor, critic, actor_optimizer, critic_optimizer)

if __name__ == '__main__':
    main()


/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode 0, Actor Loss: 13.180694580078125, Critic Loss: 476.2232666015625
Episode 1, Actor Loss: 3.101984977722168, Critic Loss: 35.90409469604492
Episode 2, Actor Loss: 2.794888973236084, Critic Loss: 28.875808715820312
Episode 3, Actor Loss: 4.026198387145996, Critic Loss: 55.305179595947266
Episode 4, Actor Loss: 9.746953010559082, Critic Loss: 266.1950378417969
Episode 5, Actor Loss: 4.7715654373168945, Critic Loss: 71.94722747802734
Episode 6, Actor Loss: 3.556051015853882, Critic Loss: 48.121334075927734
Episode 7, Actor Loss: 4.782294750213623, Critic Loss: 72.4375
Episode 8, Actor Loss: 4.582406044006348, Critic Loss: 65.80677795410156
Episode 9, Actor Loss: 2.62186598777771, Critic Loss: 29.442399978637695
Episode 10, Actor Loss: 4.181180477142334, Critic Loss: 57.22575759887695
Episode 11, Actor Loss: 6.029847621917725, Critic Loss: 73.16222381591797
Episode 12, Actor Loss: 2.4514853954315186, Critic Loss: 29.501253128051758
Episode 13, Actor Loss: 8.558777809143066, Critic L